In [2]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

def hwt(x):
    # Haar Wavelet Transform
    N = len(x)
    M = N // 2
    cA = [0] * M
    cD = [0] * M

    for k in range(1, M + 1):
        cA[k - 1] = x[2 * k - 2] * 1/2 + x[2 * k - 1] * 1/2
        cD[k - 1] = abs(x[2 * k - 2] - x[2 * k - 1])

    return cA, cD

def embed(x1, x2, bit):
    x_avg = (x1 + x2) / 2
    if bit == 0:
        x1_new = x_avg - 1
        x2_new = x_avg + 1
    else:
        x1_new = x_avg + 1
        x2_new = x_avg - 1
    return x1_new, x2_new

def recover(x1, x2):
    x_avg = (x1 + x2) / 2
    bit = 0 if x1 > x_avg else 1
    return x1, x2, bit

def caculate_T(bits_length, x):
    return bits_length / (2 * x.shape[1])

def imtobinary(image_path):
    image = plt.imread(image_path)
    binary_image = (image > 0.5).astype(int)
    return binary_image.flatten()

def myimshow(image, title):
    plt.imshow(image, cmap='gray')
    plt.title(title)
    plt.show()

# ----------STEP1: Haar Wavelet Transform------------
print('----------STEP1: Haar Wavelet Transform------------')
x_original = np.array(Image.open("images/dog.png").convert("L"))
row, col = x_original.shape
x = x_original.astype(float)

# Horizontal
x1 = np.zeros_like(x)
for j in range(row):
    tmp1 = x[j, :]
    L, H = hwt(tmp1)
    x1[j, :] = np.concatenate((L, H))

# Vertical
x2 = np.zeros_like(x1)
for i in range(col):
    tmp1 = x[:, i]
    ra1, rd1 = hwt(tmp1)
    x2[:, i] = np.concatenate((ra1, rd1))

x2 = x2.astype(np.uint8)

----------STEP1: Haar Wavelet Transform------------


In [17]:
# ----------STEP2: Watermark to Binary------------
print('----------STEP2: Watermark to Binary------------')
watermark = [0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

# ----------STEP3: Embedding------------
print('----------STEP3: Embedding------------')
M = col // 2
T = caculate_T(len(watermark), x)

# Mark ID
count_ne_bar = 0
count_u = 0
count_m = 0
count_ne = 0
count_n = 0
count_total = 0
ID = np.zeros((row, M), dtype=int)

for i in range(row):
    for k in range(M):
        count_total += 1
        h = abs(x[i, 2 * k - 1] - x[i, 2 * k])
        avg_intensity = (x[i, 2 * k - 1] + x[i, 2 * k]) / 2

        if h > 2 * T + 1 or avg_intensity >= 255 - T or avg_intensity < T:
            count_u += 1
            ID[i, k] = 1  # u
        elif h > np.floor((T - 1) / 2):
            count_n += 1
            if h > np.floor(T):
                count_ne_bar += 1
                ID[i, k] = 2  # ne_bar
            else:
                count_ne += 1
                ID[i, k] = 3  # ne
        else:
            count_m += 1
            ID[i, k] = 4  # m

# Draw the Map
map = np.zeros(count_n, dtype=int)
count = 0
for i in range(row):
    for k in range(M):
        if ID[i, k] == 2:
            count += 1
        elif ID[i, k] == 3:
            map[count] = 1
            count += 1

# Combine the encoded values into a single stream
encoded_information_stream = format(col // 2, '08b') +format(9, '08b') + format(42, '08b')  # encoded identity
encoded_information_stream

----------STEP2: Watermark to Binary------------
----------STEP3: Embedding------------


'1000000000000100100101010'

In [1]:
# Embed the information stream into the last pixels using LSB replacement
num_bits_to_embed = len(encoded_information_stream)
last_pixels = x2.flatten()[-num_bits_to_embed:]

last_pixels = np.array(last_pixels, dtype=int)

for i, bit in enumerate(encoded_information_stream):
    last_pixels[i] &= 0b11111110  # Clear LSB
    last_pixels[i] |= int(bit)

# Extract the original LSBs
original_LSBs = [byte & 0b00000001 for byte in last_pixels]

# Combine the original LSBs with the compressed bitmap to construct overflow information O
overflow_information = np.concatenate((original_LSBs, map))

# Watermark Embedding
count = 0
for i in range(row):
    for k in range(M):
        if ID[i, k] == 4:  # If that is in SET M
            x[i, 2 * k - 1], x[i, 2 * k] = embed(x[i, 2 * k - 1], x[i, 2 * k], watermark[count])
            count += 1

count = 0
for i in range(row):
    for k in range(M):
        if ID[i, k] == 3:  # If that is in SET Ne
            x[i, 2 * k - 1], x[i, 2 * k] = embed(x[i, 2 * k - 1], x[i, 2 * k], watermark[count])
            count += 1

# ----------STEP4: Compute the Histogram------------
print('----------STEP4: Compute the Histogram------------')
plt.subplot(2, 1, 1)
plt.hist(x_original.flatten(), bins=256, range=(0, 256), density=True, color='blue', alpha=0.7)
plt.title('Original Histogram')

plt.subplot(2, 1, 2)
plt.hist(x.flatten(), bins=256, range=(0, 256), density=True, color='red', alpha=0.7)
plt.title('Watermarked Image Histogram')


NameError: name 'encoded_information_stream' is not defined

In [ ]:
# ----------STEP5: Decoding------------
print('----------STEP5: Decoding------------')
d_row, d_col = x.shape
M = d_col // 2
length_map = count_n
length_watermark = 200 * 100

# Mark the ID
d_count_ne_bar = 0
d_count_u = 0
d_count_m = 0
d_count_ne = 0
d_count_n = 0
d_count_total = 0
d_ID = np.zeros((d_row, M), dtype=int)

for i in range(d_row):
    for k in range(M):
        d_count_total += 1
        d_h = x[i, 2 * k - 1] - x[i, 2 * k]
        avg_intensity = (x[i, 2 * k - 1] + x[i, 2 * k]) / 2

        if abs(d_h) > 2 * T + 1 or avg_intensity >= 255 - T or avg_intensity < T:
            d_count_u += 1
            d_ID[i, k] = 1
        elif abs(d_h) >= 2 * np.floor(-(T - 1) / 2) and abs(d_h) <= 2 * np.floor((T - 1) / 2) + 1:
            d_count_m += 1
            d_ID[i, k] = 4
        else:
            d_count_n += 1
            d_ID[i, k] = 5

# Draw the location map from M
d_count = 0
d_map = np.zeros(d_count_n, dtype=int)
for i in range(d_row):
    for k in range(M):
        if d_h % 2 == 0 and d_count <= d_count_n:
            d_map[d_count] = 0
            d_count += 1
        if d_h % 2 == 1 and d_count <= d_count_n:
            d_map[d_count] = 1
            d_count += 1

# Use map to recover Ne and Ne_bar
d_count = 0
for i in range(d_row):
    for k in range(M):
        if d_ID[i, k] == 5:
            if d_map[d_count] == 0:
                d_ID[i, k] = 2
                d_count_ne_bar = d_count_ne_bar + 1
                d_count += 1
            elif d_map[d_count] == 1:
                d_ID[i, k] = 3
                d_count += 1
                d_count_ne = d_count_ne + 1

# Recover the data
count = 0
d_payload = np.zeros(d_count_n + length_watermark, dtype=int)
for i in range(row):
    for k in range(M):
        if ID[i, k] == 4:
            x[i, 2 * k - 1], x[i, 2 * k], d_payload[count] = recover(x[i, 2 * k - 1], x[i, 2 * k])
            count += 1

for i in range(row):
    for k in range(M):
        if ID[i, k] == 3:
            x[i, 2 * k - 1], x[i, 2 * k], d_payload[count] = recover(x[i, 2 * k - 1], x[i, 2 * k])
            count += 1

d_watermark = d_payload[d_count_n + 1:d_count_n + length_watermark + 1].reshape(100, 200)

# ----------STEP6: Comparing------------
print('----------STEP6: Comparing------------')
plt.figure()
plt.subplot(2, 1, 1)
plt.hist(original_watermark.flatten(), bins=256, range=(0, 256), density=True, color='blue', alpha=0.7)
plt.title('Original Watermark Histogram')